In [1]:
from dotenv import load_dotenv

load_dotenv()

True

In [8]:
from mcp import ClientSession, StdioServerParameters
from mcp.client.stdio import stdio_client
from langchain_ollama import ChatOllama
from langchain_mcp_adapters.tools import load_mcp_tools
from langgraph.prebuilt import create_react_agent
from langchain_groq import ChatGroq

In [12]:
model = ChatGroq(model="llama-3.3-70b-versatile",temperature=0.5)
model = ChatOllama(model="llama3.2:1b",temperature=0.0,max_new_tokens=500)

In [4]:
server_params = StdioServerParameters(
    command="python",
    # Make sure to update to the full absolute path to your weather.py file
    args=["weather.py"],
)

In [27]:

async def run_app(user_question):
    async with stdio_client(server_params) as (read, write):
        async with ClientSession(read, write) as session:
            # Initialize the connection
            await session.initialize()

            # Get tools
            tools = await load_mcp_tools(session)
            final_prompt = PromptTemplate(template="""Decide whether the {query} provided is related to math or weather.
                                          Please provide only one word as response either 'Math' or 'Weather' and no preamble explaining the response.""",
                                          input_variables=["query"])
            chain = final_prompt | model
            res_type = chain.invoke({"query":user_question})
            print(res_type.content)
            # Create and run the agent
            agent = create_react_agent(model, tools)
            if res_type.content == 'Math':
                agent_response = await agent.ainvoke({"messages": f"{user_question}"})
            else:
                agent_response = await agent.ainvoke({"messages": f"{user_question}"})
            #print(agent_response)
            print(agent_response['messages'][-1].content)

In [32]:
import asyncio


In [33]:
user_question = "what is the weather in nyc?"
await run_app(user_question=user_question)

C:\Users\nayak\AppData\Local\Programs\Python\Python312\Lib\ast.py:406: RuntimeWarning: coroutine 'run_app' was never awaited
  visitor = getattr(self, method, self.generic_visit)


NotImplementedError: 

In [5]:
from langchain_mcp_adapters.client import MultiServerMCPClient

In [13]:
async with MultiServerMCPClient(
    {
        "weather": {
            "url": "http://localhost:8000/sse",
            "transport": "sse",
        }
    }
) as client:
    agent = create_react_agent(model, client.get_tools())
    agent_response = await agent.ainvoke({"messages": "what is the weather in nyc?"})

In [18]:
print(agent_response['messages'][-1].content)

Here are the key points from the weather forecast:

**Current Weather Conditions**

* Heavy rain is expected in parts of New York, Vermont, and Massachusetts.
* Snowmelt is causing rivers and streams to rise rapidly.

**Flood Watches**

* A Flood Watch has been issued for several counties in New York, including:
	+ Oswego
	+ Jefferson
	+ Lewis
	+ Black River
	+ Jefferson, Lewis, and Oswego counties in Vermont
* A Flood Watch has also been issued for parts of Massachusetts, including:
	+ Eastern Franklin
	+ Southeastern St. Lawrence
	+ Southern Franklin
	+ Western Essex
	+ Western Clinton
	+ Western Addison
	+ Orange
	+ Western Rutland
	+ Eastern Franklin
	+ Eastern Chittenden
	+ Eastern Addison
	+ Eastern Rutland
	+ Western Windsor
	+ Eastern Windsor

**Flood Risks**

* Excessive runoff is expected to cause flooding in areas with poor drainage and urban areas.
* River ice movement is likely, which may result in isolated ice jams and associated flooding.

**Special Weather Statements**


In [15]:
from IPython.display import display, Markdown
from langchain_core.messages import HumanMessage, ToolMessage, AIMessage

In [16]:
for response in agent_response["messages"]:
    user = ""
    
    if isinstance(response, HumanMessage):
        user = "**User**"
    elif isinstance(response, ToolMessage):
        user = "**Tool**"
    elif isinstance(response, AIMessage):
        user = "**AI**"
        
    if isinstance(response.content, list):
        display(Markdown(f'{user}: {response.content[0].get("text", "")}'))
        continue
    display(Markdown(f"{user}: {response.content}"))

**User**: what is the weather in nyc?

**AI**: 

**Tool**: 
Event: Severe Thunderstorm Warning
Area: Allegany, NY; Cattaraugus, NY
Severity: Severe
Description: At 225 PM EDT, a severe thunderstorm was located near Ashford, or 17
miles northeast of Salamanca, moving northeast at 55 mph.

HAZARD...60 mph wind gusts.

SOURCE...Radar indicated.

IMPACT...Expect damage to roofs, siding, and trees.

Locations impacted include...
Vandalia, Carrollton, East Randolph, Onoville, Limestone,
Ellicottville, Ashford, Great Valley, New Albion, and Hardy Corners.

This includes Interstate 86 between exits 17 and 28.
Instructions: For your protection move to an interior room on the lowest floor of a
building.

---

Event: Severe Thunderstorm Warning
Area: Erie, NY; Genesee, NY; Niagara, NY; Orleans, NY; Wyoming, NY
Severity: Severe
Description: SVRBUF

The National Weather Service in Buffalo has issued a

* Severe Thunderstorm Warning for...
Wyoming County in western New York...
Erie County in western New York...
Niagara County in western New York...
Orleans County in western New York...
Genesee County in western New York...

* Until 315 PM EDT.

* At 218 PM EDT, severe thunderstorms were located along a line
extending from 9 miles northwest of Fort Niagara State Park to 8
miles southwest of Grand Island to 6 miles northwest of Silver
Creek, moving northeast at 65 mph.

HAZARD...60 mph wind gusts and quarter size hail.

SOURCE...Radar indicated.

IMPACT...Hail damage to vehicles is expected. Expect wind damage
to roofs, siding, and trees.

* Locations impacted include...
Angola, Clarence Center, Fort Niagara State Park, Bliss, Depew,
Colden, Newfane, Pembroke, Woodlawn Beach State Park, and Lewiston.

This includes Interstate 90 between exits 48 and 57A.
Instructions: For your protection move to an interior room on the lowest floor of a
building.

---

Event: Wind Advisory
Area: Eastern Clinton; Western Clinton; Western Essex; Eastern Essex; Grand Isle; Western Franklin; Western Chittenden; Western Addison; Western Rutland; Eastern Chittenden; Eastern Addison
Severity: Moderate
Description: * WHAT...South winds 15 to 25 mph with gusts up to 55 mph.

* WHERE...Portions of northern New York and western Vermont.

* WHEN...Until 10 PM EDT this evening.

* IMPACTS...Gusty winds will blow around unsecured objects. Tree
limbs could be blown down and a few power outages may result.
Instructions: Winds this strong can make driving difficult, especially for high
profile vehicles. Use extra caution.

Secure outdoor objects.

---

Event: Wind Advisory
Area: Northern St. Lawrence; Northern Franklin; Southeastern St. Lawrence; Southern Franklin; Southwestern St. Lawrence
Severity: Moderate
Description: * WHAT...South winds 15 to 25 mph with gusts up to 55 mph expected.

* WHERE...Franklin and St. Lawrence Counties in New York.

* WHEN...Until 10 PM EDT this evening.

* IMPACTS...Gusty winds will blow around unsecured objects. Tree
limbs could be blown down and a few power outages may result.
Instructions: Winds this strong can make driving difficult, especially for high
profile vehicles. Use extra caution.

Secure outdoor objects.

---

Event: Wind Advisory
Area: Wyoming; Livingston; Ontario; Cattaraugus; Allegany
Severity: Moderate
Description: * WHAT...South winds 20 to 30 mph with gusts up to 50 mph expected.

* WHERE...Wyoming, Livingston, Ontario, Cattaraugus, and Allegany
Counties. Strongest wind gusts will likely be over and just north
of the higher hilltops, and also near the north end of the Finger
Lakes.

* WHEN...Until 5 PM EDT this afternoon.

* IMPACTS...Gusty winds will blow around unsecured objects. Tree
limbs could be blown down and a few power outages may result.
Instructions: Use extra caution when driving, especially if operating a high
profile vehicle. Secure outdoor objects.

---

Event: Wind Advisory
Area: Monroe; Wayne
Severity: Moderate
Description: * WHAT...South winds 20 to 30 mph with gusts up to 50 mph.

* WHERE...Monroe and Wayne Counties.

* WHEN...Until 5 PM EDT this afternoon.

* IMPACTS...Gusty winds will blow around unsecured objects. Tree
limbs could be blown down and a few isolated power outages may
result.
Instructions: Use extra caution when driving, especially if operating a high
profile vehicle. Secure outdoor objects.

---

Event: Wind Advisory
Area: Niagara; Orleans; Northern Erie; Genesee
Severity: Moderate
Description: * WHAT...South winds 20 to 30 mph with gusts up to 50 mph, becoming
southwest later this afternoon.

* WHERE...Niagara, Orleans, Northern Erie, and Genesee Counties.

* WHEN...Until 8 PM EDT this evening.

* IMPACTS...Gusty winds will blow around unsecured objects. Tree
limbs could be blown down and a few isolated power outages may
result.
Instructions: Use extra caution when driving, especially if operating a high
profile vehicle. Secure outdoor objects.

---

Event: Wind Advisory
Area: Northern Cayuga; Oswego
Severity: Moderate
Description: * WHAT...South winds 20 to 30 mph with gusts up to 50 mph.

* WHERE...Northern Cayuga and Oswego Counties.

* WHEN...Until 8 PM EDT this evening.

* IMPACTS...Gusty winds will blow around unsecured objects. Tree
limbs could be blown down and a few isolated power outages may
result.
Instructions: Use extra caution when driving, especially if operating a high
profile vehicle. Secure outdoor objects.

---

Event: Wind Advisory
Area: Jefferson; Lewis
Severity: Moderate
Description: * WHAT...South winds 20 to 30 mph with gusts up to 55 mph expected.

* WHERE...Jefferson and Lewis Counties. Strongest wind gusts will
likely be on the northern slopes of the Tug Hill Plateau to near
Fort Drum.

* WHEN...Until 8 PM EDT this evening.

* IMPACTS...Gusty winds will blow around unsecured objects. Tree
limbs could be blown down and a few power outages may result.
Instructions: Use extra caution when driving, especially if operating a high
profile vehicle. Secure outdoor objects.

---

Event: Severe Thunderstorm Watch
Area: Allegany, NY; Cattaraugus, NY; Chautauqua, NY; Erie, NY; Genesee, NY; Livingston, NY; Monroe, NY; Niagara, NY; Ontario, NY; Orleans, NY; Wayne, NY; Wyoming, NY
Severity: Severe
Description: THE NATIONAL WEATHER SERVICE HAS ISSUED SEVERE THUNDERSTORM WATCH
55 IN EFFECT UNTIL 7 PM EDT THIS EVENING FOR THE FOLLOWING AREAS

IN NEW YORK THIS WATCH INCLUDES 12 COUNTIES

IN WESTERN NEW YORK

ALLEGANY              CATTARAUGUS           CHAUTAUQUA
ERIE                  GENESEE               LIVINGSTON
MONROE                NIAGARA               ONTARIO
ORLEANS               WAYNE                 WYOMING

THIS INCLUDES THE CITIES OF BATAVIA, BUFFALO, CANANDAIGUA,
GENESEO, JAMESTOWN, MEDINA, NEWARK, NIAGARA FALLS, OLEAN,
ROCHESTER, WARSAW, AND WELLSVILLE.
Instructions: None

---

Event: Severe Thunderstorm Watch
Area: Schuyler, NY; Seneca, NY; Steuben, NY; Yates, NY
Severity: Severe
Description: THE NATIONAL WEATHER SERVICE HAS ISSUED SEVERE THUNDERSTORM WATCH
55 IN EFFECT UNTIL 7 PM EDT THIS EVENING FOR THE FOLLOWING AREAS

IN NEW YORK THIS WATCH INCLUDES 4 COUNTIES

IN CENTRAL NEW YORK

SCHUYLER              SENECA                STEUBEN
YATES

THIS INCLUDES THE CITIES OF CORNING, HORNELL, PENN YAN,
SENECA FALLS, AND WATKINS GLEN.
Instructions: None

---

Event: Wind Advisory
Area: Schoharie; Western Greene; Western Ulster
Severity: Moderate
Description: * WHAT...South winds 15 to 25 mph with gusts up to 55 mph.

* WHERE...Schoharie, Western Greene, and Western Ulster Counties.

* WHEN...Until 8 PM EDT this evening.

* IMPACTS...Gusty winds will blow around unsecured objects. Tree
limbs could be blown down and a few power outages may result.
Instructions: Winds this strong can make driving difficult, especially for high
profile vehicles. Use extra caution.

---

Event: Flood Watch
Area: Northern Oneida; Southern Oneida
Severity: Severe
Description: Heavy rain may fall on a deep primed snowpack leading to the melt
increasing. Flows in rivers may increase quickly and reach critical
levels.

* WHAT...Flooding caused by rain and snowmelt continues to be
possible.

* WHERE...A portion of central New York, including the following
areas, Northern Oneida and Southern Oneida.

* WHEN...Through Monday evening.

* IMPACTS...Excessive runoff may result in flooding of rivers,
creeks, streams, and other low-lying and flood-prone locations.
Creeks and streams may rise out of their banks.

* ADDITIONAL DETAILS...
- Up to an inch of rain is possible today into this evening.
However, significant snowmelt will be an even bigger factor,
as warm weather releases a lot of water from the heavy
snowpack across the Tug Hill region. Combined snowmelt and
rain, as well as lingering ice in some streams, will pose a
significant risk for impactful flooding. The highest concern
are the branches of Fish Creek, especially from the
confluence downstream into Oneida Lake including Sylvan
Beach. Tributaries of the Mohawk River upstream of Delta
Lake, are also of higher concern.
- https://www.weather.gov/safety/flood
Instructions: You should monitor later forecasts and be alert for possible Flood
Warnings. Those living in areas prone to flooding should be prepared
to take action should flooding develop.

---

Event: Wind Advisory
Area: Yates; Seneca; Southern Cayuga; Onondaga; Steuben; Schuyler; Chemung; Tompkins; Madison; Southern Oneida; Cortland; Tioga; Bradford; Wyoming
Severity: Moderate
Description: * WHAT...South winds 20 to 30 mph with gusts up to 55 mph.

* WHERE...In New York, Chemung, Cortland, Madison, Onondaga,
Schuyler, Seneca, Southern Cayuga, Southern Oneida, Steuben,
Tioga, Tompkins, and Yates Counties. In Pennsylvania, Bradford and
Wyoming Counties.

* WHEN...Until 5 PM EDT this afternoon.

* IMPACTS...Gusty winds will blow around unsecured objects. Tree
limbs could be blown down and a few power outages may result.
Instructions: Winds this strong can make driving difficult, especially for high
profile vehicles. Use extra caution.

---

Event: Wind Advisory
Area: Chenango; Otsego; Broome; Delaware; Sullivan; Susquehanna; Northern Wayne; Lackawanna; Luzerne; Pike; Southern Wayne
Severity: Moderate
Description: * WHAT...South winds 20 to 30 mph with gusts up to 55 mph.

* WHERE...In New York, Broome, Chenango, Delaware, Otsego, and
Sullivan Counties. In Pennsylvania, Lackawanna, Luzerne, Northern
Wayne, Pike, Southern Wayne, and Susquehanna Counties.

* WHEN...Until 8 PM EDT this evening.

* IMPACTS...Gusty winds will blow around unsecured objects. Tree
limbs could be blown down and a few power outages may result.
Instructions: Winds this strong can make driving difficult, especially for high
profile vehicles. Use extra caution.

---

Event: Flood Watch
Area: Oswego; Jefferson; Lewis
Severity: Severe
Description: Heavy rain may fall on a deep primed snowpack leading to the melt
increasing. Flows in rivers may increase quickly and reach critical
levels.

* WHAT...Flooding caused by rain and snowmelt continues to be
possible.

* WHERE...A portion of central New York, including the following
counties, Jefferson, Lewis and Oswego. Flooding is most likely
along the Black River, and is also possible along creeks and
streams where snow pack still remains.

* WHEN...Through Monday evening.

* IMPACTS...Excessive runoff may result in flooding of rivers,
creeks, streams, and other low-lying and flood-prone locations. If
flooding develops, the first impacts would be along smaller
streams and in low lying areas late this afternoon through
tonight, with larger rivers possibly rising to flood stage Monday.
There is also a low chance of flooding from ice jams in areas
where significant ice remains.

* ADDITIONAL DETAILS...
- Periods of rain may produce 0.75 to 1 inches of rain today
through tonight, with the greatest rainfall amounts likely
overlapping areas with significant snowpack across the Tug
Hill Plateau and western foothills of the Adirondacks.
- http://www.weather.gov/safety/flood
Instructions: You should monitor later forecasts and be alert for possible Flood
Warnings. Those living in areas prone to flooding should be prepared
to take action should flooding develop.

---

Event: Flood Watch
Area: Northern Herkimer; Hamilton; Southern Herkimer; Southern Fulton; Montgomery; Northern Saratoga; Northern Warren; Northern Washington; Northern Fulton; Southeast Warren; Southern Washington; Bennington; Western Windham; Eastern Windham
Severity: Severe
Description: * WHAT...Flooding caused by rain and/or snowmelt and/or ice jam
continues to be possible.

* WHERE...In eastern New York, Hamilton, Montgomery, Fulton,
Herkimer, northern Saratoga, Warren, Washington Counties and in
southern Vermont, Bennington and Windham Counties.

* WHEN...From noon EDT today through Monday afternoon.

* IMPACTS...Excessive runoff may result in flooding of rivers,
creeks, streams, and other low-lying and flood-prone locations.
Flooding may occur in poor drainage and urban areas.

* ADDITIONAL DETAILS...
- Periods of rain are expected to produce 1 to 1.5 inches of
rainfall this afternoon into Monday. The rainfall combined
with significant snow melt from the snowpack in the southwest
Adirondacks and the southern Greens may result in some
flooding. Also, some ice break-up or movement may cause
flooding. Locations most vulnerable to flooding will be near
the western Mohawk and Sacandaga Rivers as well as the West
Canada Creek.
- http://www.weather.gov/safety/flood
Instructions: You should monitor later forecasts and be alert for possible Flood
Warnings. Those living in areas prone to flooding should be prepared
to take action should flooding develop.

---

Event: Flood Watch
Area: Northern St. Lawrence; Northern Franklin; Eastern Clinton; Southeastern St. Lawrence; Southern Franklin; Western Clinton; Western Essex; Eastern Essex; Southwestern St. Lawrence; Grand Isle; Western Franklin; Orleans; Essex; Western Chittenden; Lamoille; Caledonia; Washington; Western Addison; Orange; Western Rutland; Eastern Franklin; Eastern Chittenden; Eastern Addison; Eastern Rutland; Western Windsor; Eastern Windsor
Severity: Severe
Description: * WHAT...Flooding caused by rain and snowmelt continues to be
possible.

* WHERE...All of northern New York and northern and central Vermont.

* WHEN...From 8 AM EDT this morning through Monday evening.

* IMPACTS...Excessive runoff may result in flooding of rivers,
creeks, streams, and other low-lying and flood-prone locations.
River ice movement is likely, which may result in isolated ice
jams and associated flooding.

* ADDITIONAL DETAILS...
- Runoff from snowmelt due to warm temperatures will combine
with widespread rainfall to produce sharp rises on area
rivers and streams. Flooding is possible, both of larger
mainstem rivers and smaller, fast-responding mountain
streams. In addition, river ice movement is likely, which may
result in isolated ice jams and associated flooding.
- http://www.weather.gov/safety/flood
Instructions: You should monitor later forecasts and be alert for possible Flood
Warnings. Those living in areas prone to flooding should be prepared
to take action should flooding develop.


**AI**: Here are the key points from the weather forecast:

**Current Weather Conditions**

* Heavy rain is expected in parts of New York, Vermont, and Massachusetts.
* Snowmelt is causing rivers and streams to rise rapidly.

**Flood Watches**

* A Flood Watch has been issued for several counties in New York, including:
	+ Oswego
	+ Jefferson
	+ Lewis
	+ Black River
	+ Jefferson, Lewis, and Oswego counties in Vermont
* A Flood Watch has also been issued for parts of Massachusetts, including:
	+ Eastern Franklin
	+ Southeastern St. Lawrence
	+ Southern Franklin
	+ Western Essex
	+ Western Clinton
	+ Western Addison
	+ Orange
	+ Western Rutland
	+ Eastern Franklin
	+ Eastern Chittenden
	+ Eastern Addison
	+ Eastern Rutland
	+ Western Windsor
	+ Eastern Windsor

**Flood Risks**

* Excessive runoff is expected to cause flooding in areas with poor drainage and urban areas.
* River ice movement is likely, which may result in isolated ice jams and associated flooding.

**Special Weather Statements**

* A Flood Warning has been issued for the following areas:
	+ Black River
	+ Jefferson, Lewis, and Oswego counties in Vermont
	+ Eastern Franklin
	+ Southeastern St. Lawrence
	+ Southern Franklin
	+ Western Essex
	+ Western Clinton
	+ Western Addison
	+ Orange
	+ Western Rutland
	+ Eastern Franklin
	+ Eastern Chittenden
	+ Eastern Addison
	+ Eastern Rutland
	+ Western Windsor
	+ Eastern Windsor

**Next Update**

* The next update will be issued by 8 AM EDT this morning.

Please note that the forecast is subject to change, and it's always a good idea to check back for updates.

### llama-3.3-70b-versatile results

In [24]:
for response in agent_response["messages"]:
    user = ""
    
    if isinstance(response, HumanMessage):
        user = "**User**"
    elif isinstance(response, ToolMessage):
        user = "**Tool**"
    elif isinstance(response, AIMessage):
        user = "**AI**"
        
    if isinstance(response.content, list):
        display(Markdown(f'{user}: {response.content[0].get("text", "")}'))
        continue
    display(Markdown(f"{user}: {response.content}"))

**User**: what is the weather in nyc?

**AI**: 

**Tool**: 
This Afternoon:
Temperature: 60°F
Wind: 17 to 21 mph S
Forecast: A slight chance of drizzle before 5pm, then a chance of rain showers. Cloudy. High near 60, with temperatures falling to around 58 in the afternoon. South wind 17 to 21 mph, with gusts as high as 36 mph. Chance of precipitation is 30%.

---

Tonight:
Temperature: 53°F
Wind: 13 to 23 mph S
Forecast: A chance of rain showers before 8pm, then rain and isolated thunderstorms. Cloudy. Low around 53, with temperatures rising to around 55 overnight. South wind 13 to 23 mph, with gusts as high as 36 mph. Chance of precipitation is 100%. New rainfall amounts between 1 and 2 inches possible.

---

Monday:
Temperature: 56°F
Wind: 7 to 15 mph W
Forecast: Rain before 2pm, then a chance of rain showers. Cloudy. High near 56, with temperatures falling to around 49 in the afternoon. West wind 7 to 15 mph. Chance of precipitation is 90%. New rainfall amounts between a quarter and half of an inch possible.

---

Monday Night:
Temperature: 39°F
Wind: 15 mph NW
Forecast: A chance of rain showers before 8pm. Mostly cloudy, with a low around 39. Northwest wind around 15 mph. Chance of precipitation is 40%.

---

Tuesday:
Temperature: 63°F
Wind: 8 to 13 mph N
Forecast: Sunny, with a high near 63. North wind 8 to 13 mph.


**AI**: 

**Tool**: 
Event: Severe Thunderstorm Warning
Area: Erie, NY; Genesee, NY; Niagara, NY; Orleans, NY; Wyoming, NY
Severity: Severe
Description: SVRBUF

The National Weather Service in Buffalo has issued a

* Severe Thunderstorm Warning for...
Wyoming County in western New York...
Erie County in western New York...
Niagara County in western New York...
Orleans County in western New York...
Genesee County in western New York...

* Until 315 PM EDT.

* At 218 PM EDT, severe thunderstorms were located along a line
extending from 9 miles northwest of Fort Niagara State Park to 8
miles southwest of Grand Island to 6 miles northwest of Silver
Creek, moving northeast at 65 mph.

HAZARD...60 mph wind gusts and quarter size hail.

SOURCE...Radar indicated.

IMPACT...Hail damage to vehicles is expected. Expect wind damage
to roofs, siding, and trees.

* Locations impacted include...
Angola, Clarence Center, Fort Niagara State Park, Bliss, Depew,
Colden, Newfane, Pembroke, Woodlawn Beach State Park, and Lewiston.

This includes Interstate 90 between exits 48 and 57A.
Instructions: For your protection move to an interior room on the lowest floor of a
building.

---

Event: Severe Thunderstorm Warning
Area: Allegany, NY; Cattaraugus, NY
Severity: Severe
Description: At 215 PM EDT, a severe thunderstorm was located near Arcade, or 20
miles south of East Aurora, moving northeast at 70 mph.

HAZARD...60 mph wind gusts.

SOURCE...Radar indicated.

IMPACT...Expect damage to roofs, siding, and trees.

Locations impacted include...
Vandalia, Carrollton, East Randolph, Onoville, Limestone,
Ellicottville, Ashford, Great Valley, New Albion, and Hardy Corners.

This includes Interstate 86 between exits 17 and 28.
Instructions: For your protection move to an interior room on the lowest floor of a
building.

---

Event: Wind Advisory
Area: Eastern Clinton; Western Clinton; Western Essex; Eastern Essex; Grand Isle; Western Franklin; Western Chittenden; Western Addison; Western Rutland; Eastern Chittenden; Eastern Addison
Severity: Moderate
Description: * WHAT...South winds 15 to 25 mph with gusts up to 55 mph.

* WHERE...Portions of northern New York and western Vermont.

* WHEN...Until 10 PM EDT this evening.

* IMPACTS...Gusty winds will blow around unsecured objects. Tree
limbs could be blown down and a few power outages may result.
Instructions: Winds this strong can make driving difficult, especially for high
profile vehicles. Use extra caution.

Secure outdoor objects.

---

Event: Wind Advisory
Area: Northern St. Lawrence; Northern Franklin; Southeastern St. Lawrence; Southern Franklin; Southwestern St. Lawrence
Severity: Moderate
Description: * WHAT...South winds 15 to 25 mph with gusts up to 55 mph expected.

* WHERE...Franklin and St. Lawrence Counties in New York.

* WHEN...Until 10 PM EDT this evening.

* IMPACTS...Gusty winds will blow around unsecured objects. Tree
limbs could be blown down and a few power outages may result.
Instructions: Winds this strong can make driving difficult, especially for high
profile vehicles. Use extra caution.

Secure outdoor objects.

---

Event: Wind Advisory
Area: Wyoming; Livingston; Ontario; Cattaraugus; Allegany
Severity: Moderate
Description: * WHAT...South winds 20 to 30 mph with gusts up to 50 mph expected.

* WHERE...Wyoming, Livingston, Ontario, Cattaraugus, and Allegany
Counties. Strongest wind gusts will likely be over and just north
of the higher hilltops, and also near the north end of the Finger
Lakes.

* WHEN...Until 5 PM EDT this afternoon.

* IMPACTS...Gusty winds will blow around unsecured objects. Tree
limbs could be blown down and a few power outages may result.
Instructions: Use extra caution when driving, especially if operating a high
profile vehicle. Secure outdoor objects.

---

Event: Wind Advisory
Area: Monroe; Wayne
Severity: Moderate
Description: * WHAT...South winds 20 to 30 mph with gusts up to 50 mph.

* WHERE...Monroe and Wayne Counties.

* WHEN...Until 5 PM EDT this afternoon.

* IMPACTS...Gusty winds will blow around unsecured objects. Tree
limbs could be blown down and a few isolated power outages may
result.
Instructions: Use extra caution when driving, especially if operating a high
profile vehicle. Secure outdoor objects.

---

Event: Wind Advisory
Area: Niagara; Orleans; Northern Erie; Genesee
Severity: Moderate
Description: * WHAT...South winds 20 to 30 mph with gusts up to 50 mph, becoming
southwest later this afternoon.

* WHERE...Niagara, Orleans, Northern Erie, and Genesee Counties.

* WHEN...Until 8 PM EDT this evening.

* IMPACTS...Gusty winds will blow around unsecured objects. Tree
limbs could be blown down and a few isolated power outages may
result.
Instructions: Use extra caution when driving, especially if operating a high
profile vehicle. Secure outdoor objects.

---

Event: Wind Advisory
Area: Northern Cayuga; Oswego
Severity: Moderate
Description: * WHAT...South winds 20 to 30 mph with gusts up to 50 mph.

* WHERE...Northern Cayuga and Oswego Counties.

* WHEN...Until 8 PM EDT this evening.

* IMPACTS...Gusty winds will blow around unsecured objects. Tree
limbs could be blown down and a few isolated power outages may
result.
Instructions: Use extra caution when driving, especially if operating a high
profile vehicle. Secure outdoor objects.

---

Event: Wind Advisory
Area: Jefferson; Lewis
Severity: Moderate
Description: * WHAT...South winds 20 to 30 mph with gusts up to 55 mph expected.

* WHERE...Jefferson and Lewis Counties. Strongest wind gusts will
likely be on the northern slopes of the Tug Hill Plateau to near
Fort Drum.

* WHEN...Until 8 PM EDT this evening.

* IMPACTS...Gusty winds will blow around unsecured objects. Tree
limbs could be blown down and a few power outages may result.
Instructions: Use extra caution when driving, especially if operating a high
profile vehicle. Secure outdoor objects.

---

Event: Severe Thunderstorm Watch
Area: Allegany, NY; Cattaraugus, NY; Chautauqua, NY; Erie, NY; Genesee, NY; Livingston, NY; Monroe, NY; Niagara, NY; Ontario, NY; Orleans, NY; Wayne, NY; Wyoming, NY
Severity: Severe
Description: THE NATIONAL WEATHER SERVICE HAS ISSUED SEVERE THUNDERSTORM WATCH
55 IN EFFECT UNTIL 7 PM EDT THIS EVENING FOR THE FOLLOWING AREAS

IN NEW YORK THIS WATCH INCLUDES 12 COUNTIES

IN WESTERN NEW YORK

ALLEGANY              CATTARAUGUS           CHAUTAUQUA
ERIE                  GENESEE               LIVINGSTON
MONROE                NIAGARA               ONTARIO
ORLEANS               WAYNE                 WYOMING

THIS INCLUDES THE CITIES OF BATAVIA, BUFFALO, CANANDAIGUA,
GENESEO, JAMESTOWN, MEDINA, NEWARK, NIAGARA FALLS, OLEAN,
ROCHESTER, WARSAW, AND WELLSVILLE.
Instructions: None

---

Event: Severe Thunderstorm Watch
Area: Schuyler, NY; Seneca, NY; Steuben, NY; Yates, NY
Severity: Severe
Description: THE NATIONAL WEATHER SERVICE HAS ISSUED SEVERE THUNDERSTORM WATCH
55 IN EFFECT UNTIL 7 PM EDT THIS EVENING FOR THE FOLLOWING AREAS

IN NEW YORK THIS WATCH INCLUDES 4 COUNTIES

IN CENTRAL NEW YORK

SCHUYLER              SENECA                STEUBEN
YATES

THIS INCLUDES THE CITIES OF CORNING, HORNELL, PENN YAN,
SENECA FALLS, AND WATKINS GLEN.
Instructions: None

---

Event: Wind Advisory
Area: Schoharie; Western Greene; Western Ulster
Severity: Moderate
Description: * WHAT...South winds 15 to 25 mph with gusts up to 55 mph.

* WHERE...Schoharie, Western Greene, and Western Ulster Counties.

* WHEN...Until 8 PM EDT this evening.

* IMPACTS...Gusty winds will blow around unsecured objects. Tree
limbs could be blown down and a few power outages may result.
Instructions: Winds this strong can make driving difficult, especially for high
profile vehicles. Use extra caution.

---

Event: Flood Watch
Area: Northern Oneida; Southern Oneida
Severity: Severe
Description: Heavy rain may fall on a deep primed snowpack leading to the melt
increasing. Flows in rivers may increase quickly and reach critical
levels.

* WHAT...Flooding caused by rain and snowmelt continues to be
possible.

* WHERE...A portion of central New York, including the following
areas, Northern Oneida and Southern Oneida.

* WHEN...Through Monday evening.

* IMPACTS...Excessive runoff may result in flooding of rivers,
creeks, streams, and other low-lying and flood-prone locations.
Creeks and streams may rise out of their banks.

* ADDITIONAL DETAILS...
- Up to an inch of rain is possible today into this evening.
However, significant snowmelt will be an even bigger factor,
as warm weather releases a lot of water from the heavy
snowpack across the Tug Hill region. Combined snowmelt and
rain, as well as lingering ice in some streams, will pose a
significant risk for impactful flooding. The highest concern
are the branches of Fish Creek, especially from the
confluence downstream into Oneida Lake including Sylvan
Beach. Tributaries of the Mohawk River upstream of Delta
Lake, are also of higher concern.
- https://www.weather.gov/safety/flood
Instructions: You should monitor later forecasts and be alert for possible Flood
Warnings. Those living in areas prone to flooding should be prepared
to take action should flooding develop.

---

Event: Wind Advisory
Area: Yates; Seneca; Southern Cayuga; Onondaga; Steuben; Schuyler; Chemung; Tompkins; Madison; Southern Oneida; Cortland; Tioga; Bradford; Wyoming
Severity: Moderate
Description: * WHAT...South winds 20 to 30 mph with gusts up to 55 mph.

* WHERE...In New York, Chemung, Cortland, Madison, Onondaga,
Schuyler, Seneca, Southern Cayuga, Southern Oneida, Steuben,
Tioga, Tompkins, and Yates Counties. In Pennsylvania, Bradford and
Wyoming Counties.

* WHEN...Until 5 PM EDT this afternoon.

* IMPACTS...Gusty winds will blow around unsecured objects. Tree
limbs could be blown down and a few power outages may result.
Instructions: Winds this strong can make driving difficult, especially for high
profile vehicles. Use extra caution.

---

Event: Wind Advisory
Area: Chenango; Otsego; Broome; Delaware; Sullivan; Susquehanna; Northern Wayne; Lackawanna; Luzerne; Pike; Southern Wayne
Severity: Moderate
Description: * WHAT...South winds 20 to 30 mph with gusts up to 55 mph.

* WHERE...In New York, Broome, Chenango, Delaware, Otsego, and
Sullivan Counties. In Pennsylvania, Lackawanna, Luzerne, Northern
Wayne, Pike, Southern Wayne, and Susquehanna Counties.

* WHEN...Until 8 PM EDT this evening.

* IMPACTS...Gusty winds will blow around unsecured objects. Tree
limbs could be blown down and a few power outages may result.
Instructions: Winds this strong can make driving difficult, especially for high
profile vehicles. Use extra caution.

---

Event: Flood Watch
Area: Oswego; Jefferson; Lewis
Severity: Severe
Description: Heavy rain may fall on a deep primed snowpack leading to the melt
increasing. Flows in rivers may increase quickly and reach critical
levels.

* WHAT...Flooding caused by rain and snowmelt continues to be
possible.

* WHERE...A portion of central New York, including the following
counties, Jefferson, Lewis and Oswego. Flooding is most likely
along the Black River, and is also possible along creeks and
streams where snow pack still remains.

* WHEN...Through Monday evening.

* IMPACTS...Excessive runoff may result in flooding of rivers,
creeks, streams, and other low-lying and flood-prone locations. If
flooding develops, the first impacts would be along smaller
streams and in low lying areas late this afternoon through
tonight, with larger rivers possibly rising to flood stage Monday.
There is also a low chance of flooding from ice jams in areas
where significant ice remains.

* ADDITIONAL DETAILS...
- Periods of rain may produce 0.75 to 1 inches of rain today
through tonight, with the greatest rainfall amounts likely
overlapping areas with significant snowpack across the Tug
Hill Plateau and western foothills of the Adirondacks.
- http://www.weather.gov/safety/flood
Instructions: You should monitor later forecasts and be alert for possible Flood
Warnings. Those living in areas prone to flooding should be prepared
to take action should flooding develop.

---

Event: Flood Watch
Area: Northern Herkimer; Hamilton; Southern Herkimer; Southern Fulton; Montgomery; Northern Saratoga; Northern Warren; Northern Washington; Northern Fulton; Southeast Warren; Southern Washington; Bennington; Western Windham; Eastern Windham
Severity: Severe
Description: * WHAT...Flooding caused by rain and/or snowmelt and/or ice jam
continues to be possible.

* WHERE...In eastern New York, Hamilton, Montgomery, Fulton,
Herkimer, northern Saratoga, Warren, Washington Counties and in
southern Vermont, Bennington and Windham Counties.

* WHEN...From noon EDT today through Monday afternoon.

* IMPACTS...Excessive runoff may result in flooding of rivers,
creeks, streams, and other low-lying and flood-prone locations.
Flooding may occur in poor drainage and urban areas.

* ADDITIONAL DETAILS...
- Periods of rain are expected to produce 1 to 1.5 inches of
rainfall this afternoon into Monday. The rainfall combined
with significant snow melt from the snowpack in the southwest
Adirondacks and the southern Greens may result in some
flooding. Also, some ice break-up or movement may cause
flooding. Locations most vulnerable to flooding will be near
the western Mohawk and Sacandaga Rivers as well as the West
Canada Creek.
- http://www.weather.gov/safety/flood
Instructions: You should monitor later forecasts and be alert for possible Flood
Warnings. Those living in areas prone to flooding should be prepared
to take action should flooding develop.

---

Event: Flood Watch
Area: Northern St. Lawrence; Northern Franklin; Eastern Clinton; Southeastern St. Lawrence; Southern Franklin; Western Clinton; Western Essex; Eastern Essex; Southwestern St. Lawrence; Grand Isle; Western Franklin; Orleans; Essex; Western Chittenden; Lamoille; Caledonia; Washington; Western Addison; Orange; Western Rutland; Eastern Franklin; Eastern Chittenden; Eastern Addison; Eastern Rutland; Western Windsor; Eastern Windsor
Severity: Severe
Description: * WHAT...Flooding caused by rain and snowmelt continues to be
possible.

* WHERE...All of northern New York and northern and central Vermont.

* WHEN...From 8 AM EDT this morning through Monday evening.

* IMPACTS...Excessive runoff may result in flooding of rivers,
creeks, streams, and other low-lying and flood-prone locations.
River ice movement is likely, which may result in isolated ice
jams and associated flooding.

* ADDITIONAL DETAILS...
- Runoff from snowmelt due to warm temperatures will combine
with widespread rainfall to produce sharp rises on area
rivers and streams. Flooding is possible, both of larger
mainstem rivers and smaller, fast-responding mountain
streams. In addition, river ice movement is likely, which may
result in isolated ice jams and associated flooding.
- http://www.weather.gov/safety/flood
Instructions: You should monitor later forecasts and be alert for possible Flood
Warnings. Those living in areas prone to flooding should be prepared
to take action should flooding develop.


**AI**: There are several weather alerts in effect for New York, including severe thunderstorm warnings, wind advisories, and flood watches. The specific alerts and their details are as follows:

* Severe Thunderstorm Warning: 
  - Erie, NY; Genesee, NY; Niagara, NY; Orleans, NY; Wyoming, NY
  - Allegany, NY; Cattaraugus, NY
* Wind Advisory: 
  - Western Clinton; Western Essex; Western Chittenden; Western Addison; Western Rutland; Eastern Chittenden; Eastern Addison; Eastern Rutland; Western Windsor; Eastern Windsor
  - Northern St. Lawrence; Northern Franklin; Eastern Clinton; Southeastern St. Lawrence; Southern Franklin; Western Clinton; Western Essex; Eastern Essex; Southwestern St. Lawrence; Grand Isle; Western Franklin; Orleans; Essex
  - Northern Cayuga; Oswego
  - Jefferson; Lewis
  - Wyoming; Livingston; Ontario; Cattaraugus; Allegany
  - Monroe; Wayne
  - Niagara; Orleans; Northern Erie; Genesee
  - Northern Cayuga; Oswego
  - Schoharie; Western Greene; Western Ulster
  - Yates; Seneca; Southern Cayuga; Onondaga; Steuben; Schuyler; Chemung; Tompkins; Madison; Southern Oneida; Cortland; Tioga; Bradford; Wyoming
  - Chenango; Otsego; Broome; Delaware; Sullivan; Susquehanna; Northern Wayne; Lackawanna; Luzerne; Pike; Southern Wayne
* Flood Watch: 
  - Northern Oneida; Southern Oneida
  - Oswego; Jefferson; Lewis
  - Northern Herkimer; Hamilton; Southern Herkimer; Southern Fulton; Montgomery; Northern Saratoga; Northern Warren; Northern Washington; Northern Fulton; Southeast Warren; Southern Washington; Bennington; Western Windham; Eastern Windham
  - Northern St. Lawrence; Northern Franklin; Eastern Clinton; Southeastern St. Lawrence; Southern Franklin; Western Clinton; Western Essex; Eastern Essex; Southwestern St. Lawrence; Grand Isle; Western Franklin; Orleans; Essex; Western Chittenden; Lamoille; Caledonia; Washington; Western Addison; Orange; Western Rutland; Eastern Franklin; Eastern Chittenden; Eastern Addison; Eastern Rutland; Western Windsor; Eastern Windsor

It is essential to monitor the weather forecast and warnings to stay safe and prepared for any potential weather-related hazards.

In [15]:
from IPython.display import display, Markdown
from langchain_core.messages import HumanMessage, ToolMessage, AIMessage

In [16]:

async with MultiServerMCPClient(
    {
        "weather": {
            "url": "http://localhost:8000/sse",
            "transport": "sse",
        }
    }
) as client:
    agent = create_react_agent(model, client.get_tools())
    
    # Stream the response chunks
    async for chunk in agent.astream({"messages": "what is the weather in nyc?"}):
        # Extract the message content from the AddableUpdatesDict structure
        if 'agent' in chunk and 'messages' in chunk['agent']:
            for message in chunk['agent']['messages']:
                if isinstance(message, AIMessage):
                    # Handle different content formats
                    if isinstance(message.content, list):
                        # For structured content with text and tool use
                        for item in message.content:
                            if isinstance(item, dict) and 'text' in item:
                                display(Markdown(f"**AI**: {item['text']}"))
                    else:
                        # For simple text content
                        display(Markdown(f"**AI**: {message.content}"))
                        
        elif 'tools' in chunk and 'messages' in chunk['tools']:
            for message in chunk['tools']['messages']:
                if hasattr(message, 'name') and hasattr(message, 'content'):
                    # Display tool response
                    display(Markdown(f"**Tool ({message.name})**: {message.content}"))

**AI**: 

**Tool (get_forecast)**: 
This Afternoon:
Temperature: 60°F
Wind: 17 to 21 mph S
Forecast: A slight chance of drizzle before 5pm, then a chance of rain showers. Cloudy. High near 60, with temperatures falling to around 58 in the afternoon. South wind 17 to 21 mph, with gusts as high as 36 mph. Chance of precipitation is 30%.

---

Tonight:
Temperature: 53°F
Wind: 13 to 23 mph S
Forecast: A chance of rain showers before 8pm, then rain and isolated thunderstorms. Cloudy. Low around 53, with temperatures rising to around 55 overnight. South wind 13 to 23 mph, with gusts as high as 36 mph. Chance of precipitation is 100%. New rainfall amounts between 1 and 2 inches possible.

---

Monday:
Temperature: 56°F
Wind: 7 to 15 mph W
Forecast: Rain before 2pm, then a chance of rain showers. Cloudy. High near 56, with temperatures falling to around 49 in the afternoon. West wind 7 to 15 mph. Chance of precipitation is 90%. New rainfall amounts between a quarter and half of an inch possible.

---

Monday Night:
Temperature: 39°F
Wind: 15 mph NW
Forecast: A chance of rain showers before 8pm. Mostly cloudy, with a low around 39. Northwest wind around 15 mph. Chance of precipitation is 40%.

---

Tuesday:
Temperature: 63°F
Wind: 8 to 13 mph N
Forecast: Sunny, with a high near 63. North wind 8 to 13 mph.


**AI**: 

**Tool (get_alerts)**: 
Event: Severe Thunderstorm Warning
Area: Allegany, NY; Cattaraugus, NY
Severity: Severe
Description: SVRBUF

The National Weather Service in Buffalo has issued a

* Severe Thunderstorm Warning for...
Cattaraugus County in western New York...
Western Allegany County in western New York...

* Until 245 PM EDT.

* At 149 PM EDT, a severe thunderstorm was located over Allegany
State Park, moving northeast at 70 mph.

HAZARD...60 mph wind gusts.

SOURCE...Radar indicated.

IMPACT...Expect damage to roofs, siding, and trees.

* Locations impacted include...
Vandalia, Carrollton, East Randolph, Onoville, Limestone,
Ellicottville, Ashford, Great Valley, New Albion, and Hardy
Corners.

This includes Interstate 86 between exits 17 and 28.
Instructions: For your protection move to an interior room on the lowest floor of a
building.

---

Event: Wind Advisory
Area: Wyoming; Livingston; Ontario; Cattaraugus; Allegany
Severity: Moderate
Description: * WHAT...South winds 20 to 30 mph with gusts up to 50 mph expected.

* WHERE...Wyoming, Livingston, Ontario, Cattaraugus, and Allegany
Counties. Strongest wind gusts will likely be over and just north
of the higher hilltops, and also near the north end of the Finger
Lakes.

* WHEN...Until 5 PM EDT this afternoon.

* IMPACTS...Gusty winds will blow around unsecured objects. Tree
limbs could be blown down and a few power outages may result.
Instructions: Use extra caution when driving, especially if operating a high
profile vehicle. Secure outdoor objects.

---

Event: Wind Advisory
Area: Monroe; Wayne
Severity: Moderate
Description: * WHAT...South winds 20 to 30 mph with gusts up to 50 mph.

* WHERE...Monroe and Wayne Counties.

* WHEN...Until 5 PM EDT this afternoon.

* IMPACTS...Gusty winds will blow around unsecured objects. Tree
limbs could be blown down and a few isolated power outages may
result.
Instructions: Use extra caution when driving, especially if operating a high
profile vehicle. Secure outdoor objects.

---

Event: Wind Advisory
Area: Niagara; Orleans; Northern Erie; Genesee
Severity: Moderate
Description: * WHAT...South winds 20 to 30 mph with gusts up to 50 mph, becoming
southwest later this afternoon.

* WHERE...Niagara, Orleans, Northern Erie, and Genesee Counties.

* WHEN...Until 8 PM EDT this evening.

* IMPACTS...Gusty winds will blow around unsecured objects. Tree
limbs could be blown down and a few isolated power outages may
result.
Instructions: Use extra caution when driving, especially if operating a high
profile vehicle. Secure outdoor objects.

---

Event: Wind Advisory
Area: Northern Cayuga; Oswego
Severity: Moderate
Description: * WHAT...South winds 20 to 30 mph with gusts up to 50 mph.

* WHERE...Northern Cayuga and Oswego Counties.

* WHEN...Until 8 PM EDT this evening.

* IMPACTS...Gusty winds will blow around unsecured objects. Tree
limbs could be blown down and a few isolated power outages may
result.
Instructions: Use extra caution when driving, especially if operating a high
profile vehicle. Secure outdoor objects.

---

Event: Wind Advisory
Area: Jefferson; Lewis
Severity: Moderate
Description: * WHAT...South winds 20 to 30 mph with gusts up to 55 mph expected.

* WHERE...Jefferson and Lewis Counties. Strongest wind gusts will
likely be on the northern slopes of the Tug Hill Plateau to near
Fort Drum.

* WHEN...Until 8 PM EDT this evening.

* IMPACTS...Gusty winds will blow around unsecured objects. Tree
limbs could be blown down and a few power outages may result.
Instructions: Use extra caution when driving, especially if operating a high
profile vehicle. Secure outdoor objects.

---

Event: Severe Thunderstorm Warning
Area: Cattaraugus, NY; Chautauqua, NY
Severity: Severe
Description: At 143 PM EDT, a severe thunderstorm was located 7 miles south of
Fredonia, moving northeast at 35 mph.

HAZARD...60 mph wind gusts and quarter size hail.

SOURCE...Radar indicated.

IMPACT...Hail damage to vehicles is expected. Expect wind damage to
roofs, siding, and trees.

Locations impacted include...
Silver Creek Reservoir, East Randolph, South Dayton, Onoville,
Perrysburg, Ellington, Mayville, Busti, Stow, and Long Point State
Park.

This includes the following highways...
Interstate 90 between exits 58 and 61.
Interstate 86 between exits 4 and 17.
Instructions: For your protection move to an interior room on the lowest floor of a
building.

---

Event: Severe Thunderstorm Watch
Area: Allegany, NY; Cattaraugus, NY; Chautauqua, NY; Erie, NY; Genesee, NY; Livingston, NY; Monroe, NY; Niagara, NY; Ontario, NY; Orleans, NY; Wayne, NY; Wyoming, NY
Severity: Severe
Description: THE NATIONAL WEATHER SERVICE HAS ISSUED SEVERE THUNDERSTORM WATCH
55 IN EFFECT UNTIL 7 PM EDT THIS EVENING FOR THE FOLLOWING AREAS

IN NEW YORK THIS WATCH INCLUDES 12 COUNTIES

IN WESTERN NEW YORK

ALLEGANY              CATTARAUGUS           CHAUTAUQUA
ERIE                  GENESEE               LIVINGSTON
MONROE                NIAGARA               ONTARIO
ORLEANS               WAYNE                 WYOMING

THIS INCLUDES THE CITIES OF BATAVIA, BUFFALO, CANANDAIGUA,
GENESEO, JAMESTOWN, MEDINA, NEWARK, NIAGARA FALLS, OLEAN,
ROCHESTER, WARSAW, AND WELLSVILLE.
Instructions: None

---

Event: Severe Thunderstorm Watch
Area: Schuyler, NY; Seneca, NY; Steuben, NY; Yates, NY
Severity: Severe
Description: THE NATIONAL WEATHER SERVICE HAS ISSUED SEVERE THUNDERSTORM WATCH
55 IN EFFECT UNTIL 7 PM EDT THIS EVENING FOR THE FOLLOWING AREAS

IN NEW YORK THIS WATCH INCLUDES 4 COUNTIES

IN CENTRAL NEW YORK

SCHUYLER              SENECA                STEUBEN
YATES

THIS INCLUDES THE CITIES OF CORNING, HORNELL, PENN YAN,
SENECA FALLS, AND WATKINS GLEN.
Instructions: None

---

Event: Wind Advisory
Area: Schoharie; Western Greene; Western Ulster
Severity: Moderate
Description: * WHAT...South winds 15 to 25 mph with gusts up to 55 mph.

* WHERE...Schoharie, Western Greene, and Western Ulster Counties.

* WHEN...Until 8 PM EDT this evening.

* IMPACTS...Gusty winds will blow around unsecured objects. Tree
limbs could be blown down and a few power outages may result.
Instructions: Winds this strong can make driving difficult, especially for high
profile vehicles. Use extra caution.

---

Event: Flood Watch
Area: Northern Oneida; Southern Oneida
Severity: Severe
Description: Heavy rain may fall on a deep primed snowpack leading to the melt
increasing. Flows in rivers may increase quickly and reach critical
levels.

* WHAT...Flooding caused by rain and snowmelt continues to be
possible.

* WHERE...A portion of central New York, including the following
areas, Northern Oneida and Southern Oneida.

* WHEN...Through Monday evening.

* IMPACTS...Excessive runoff may result in flooding of rivers,
creeks, streams, and other low-lying and flood-prone locations.
Creeks and streams may rise out of their banks.

* ADDITIONAL DETAILS...
- Up to an inch of rain is possible today into this evening.
However, significant snowmelt will be an even bigger factor,
as warm weather releases a lot of water from the heavy
snowpack across the Tug Hill region. Combined snowmelt and
rain, as well as lingering ice in some streams, will pose a
significant risk for impactful flooding. The highest concern
are the branches of Fish Creek, especially from the
confluence downstream into Oneida Lake including Sylvan
Beach. Tributaries of the Mohawk River upstream of Delta
Lake, are also of higher concern.
- https://www.weather.gov/safety/flood
Instructions: You should monitor later forecasts and be alert for possible Flood
Warnings. Those living in areas prone to flooding should be prepared
to take action should flooding develop.

---

Event: Wind Advisory
Area: Yates; Seneca; Southern Cayuga; Onondaga; Steuben; Schuyler; Chemung; Tompkins; Madison; Southern Oneida; Cortland; Tioga; Bradford; Wyoming
Severity: Moderate
Description: * WHAT...South winds 20 to 30 mph with gusts up to 55 mph.

* WHERE...In New York, Chemung, Cortland, Madison, Onondaga,
Schuyler, Seneca, Southern Cayuga, Southern Oneida, Steuben,
Tioga, Tompkins, and Yates Counties. In Pennsylvania, Bradford and
Wyoming Counties.

* WHEN...Until 5 PM EDT this afternoon.

* IMPACTS...Gusty winds will blow around unsecured objects. Tree
limbs could be blown down and a few power outages may result.
Instructions: Winds this strong can make driving difficult, especially for high
profile vehicles. Use extra caution.

---

Event: Wind Advisory
Area: Chenango; Otsego; Broome; Delaware; Sullivan; Susquehanna; Northern Wayne; Lackawanna; Luzerne; Pike; Southern Wayne
Severity: Moderate
Description: * WHAT...South winds 20 to 30 mph with gusts up to 55 mph.

* WHERE...In New York, Broome, Chenango, Delaware, Otsego, and
Sullivan Counties. In Pennsylvania, Lackawanna, Luzerne, Northern
Wayne, Pike, Southern Wayne, and Susquehanna Counties.

* WHEN...Until 8 PM EDT this evening.

* IMPACTS...Gusty winds will blow around unsecured objects. Tree
limbs could be blown down and a few power outages may result.
Instructions: Winds this strong can make driving difficult, especially for high
profile vehicles. Use extra caution.

---

Event: Flood Watch
Area: Oswego; Jefferson; Lewis
Severity: Severe
Description: Heavy rain may fall on a deep primed snowpack leading to the melt
increasing. Flows in rivers may increase quickly and reach critical
levels.

* WHAT...Flooding caused by rain and snowmelt continues to be
possible.

* WHERE...A portion of central New York, including the following
counties, Jefferson, Lewis and Oswego. Flooding is most likely
along the Black River, and is also possible along creeks and
streams where snow pack still remains.

* WHEN...Through Monday evening.

* IMPACTS...Excessive runoff may result in flooding of rivers,
creeks, streams, and other low-lying and flood-prone locations. If
flooding develops, the first impacts would be along smaller
streams and in low lying areas late this afternoon through
tonight, with larger rivers possibly rising to flood stage Monday.
There is also a low chance of flooding from ice jams in areas
where significant ice remains.

* ADDITIONAL DETAILS...
- Periods of rain may produce 0.75 to 1 inches of rain today
through tonight, with the greatest rainfall amounts likely
overlapping areas with significant snowpack across the Tug
Hill Plateau and western foothills of the Adirondacks.
- http://www.weather.gov/safety/flood
Instructions: You should monitor later forecasts and be alert for possible Flood
Warnings. Those living in areas prone to flooding should be prepared
to take action should flooding develop.

---

Event: Flood Watch
Area: Northern Herkimer; Hamilton; Southern Herkimer; Southern Fulton; Montgomery; Northern Saratoga; Northern Warren; Northern Washington; Northern Fulton; Southeast Warren; Southern Washington; Bennington; Western Windham; Eastern Windham
Severity: Severe
Description: * WHAT...Flooding caused by rain and/or snowmelt and/or ice jam
continues to be possible.

* WHERE...In eastern New York, Hamilton, Montgomery, Fulton,
Herkimer, northern Saratoga, Warren, Washington Counties and in
southern Vermont, Bennington and Windham Counties.

* WHEN...From noon EDT today through Monday afternoon.

* IMPACTS...Excessive runoff may result in flooding of rivers,
creeks, streams, and other low-lying and flood-prone locations.
Flooding may occur in poor drainage and urban areas.

* ADDITIONAL DETAILS...
- Periods of rain are expected to produce 1 to 1.5 inches of
rainfall this afternoon into Monday. The rainfall combined
with significant snow melt from the snowpack in the southwest
Adirondacks and the southern Greens may result in some
flooding. Also, some ice break-up or movement may cause
flooding. Locations most vulnerable to flooding will be near
the western Mohawk and Sacandaga Rivers as well as the West
Canada Creek.
- http://www.weather.gov/safety/flood
Instructions: You should monitor later forecasts and be alert for possible Flood
Warnings. Those living in areas prone to flooding should be prepared
to take action should flooding develop.

---

Event: Wind Advisory
Area: Northern St. Lawrence; Northern Franklin; Southeastern St. Lawrence; Southern Franklin; Southwestern St. Lawrence
Severity: Moderate
Description: * WHAT...South winds 15 to 25 mph with gusts up to 50 mph expected.

* WHERE...St. Lawrence and Franklin counties in northern New York.

* WHEN...Until 10 PM EDT this evening.

* IMPACTS...Gusty winds will blow around unsecured objects. Tree
limbs could be blown down and a few power outages may result.
Instructions: Winds this strong can make driving difficult, especially for high
profile vehicles. Use extra caution.

Secure outdoor objects.

---

Event: Flood Watch
Area: Northern St. Lawrence; Northern Franklin; Eastern Clinton; Southeastern St. Lawrence; Southern Franklin; Western Clinton; Western Essex; Eastern Essex; Southwestern St. Lawrence; Grand Isle; Western Franklin; Orleans; Essex; Western Chittenden; Lamoille; Caledonia; Washington; Western Addison; Orange; Western Rutland; Eastern Franklin; Eastern Chittenden; Eastern Addison; Eastern Rutland; Western Windsor; Eastern Windsor
Severity: Severe
Description: * WHAT...Flooding caused by rain and snowmelt continues to be
possible.

* WHERE...All of northern New York and northern and central Vermont.

* WHEN...From 8 AM EDT this morning through Monday evening.

* IMPACTS...Excessive runoff may result in flooding of rivers,
creeks, streams, and other low-lying and flood-prone locations.
River ice movement is likely, which may result in isolated ice
jams and associated flooding.

* ADDITIONAL DETAILS...
- Runoff from snowmelt due to warm temperatures will combine
with widespread rainfall to produce sharp rises on area
rivers and streams. Flooding is possible, both of larger
mainstem rivers and smaller, fast-responding mountain
streams. In addition, river ice movement is likely, which may
result in isolated ice jams and associated flooding.
- http://www.weather.gov/safety/flood
Instructions: You should monitor later forecasts and be alert for possible Flood
Warnings. Those living in areas prone to flooding should be prepared
to take action should flooding develop.


**AI**: I've obtained the weather forecast for New York City and weather alerts for the state of New York. Please review the information for the latest updates.